In [1]:
import pandas as pd
import jsonlines
from geopy.distance import distance
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

In [2]:
df_metro = pd.read_json("../data/input/metro.jsonlines", lines=True)
df_metro

,linha,estacao,lat,lng
0,Linha Guapimirim da SuperVia,Estação Guapimirim,-22.519500,-42.977194
1,Linha Guapimirim da SuperVia,Estação Jardim Guapimirim,-22.562586,-42.995003
2,Linha Guapimirim da SuperVia,Estação Parada Ideal,-22.579494,-43.005225
3,Linha Guapimirim da SuperVia,Estação Parada Modelo,-22.545756,-42.985000
4,Linha Japeri da SuperVia,Estação Engenheiro Pedreira,-22.679417,-43.605161
...,...,...,...,...
150,Linha Guapimirim da SuperVia,Estação Parque Estrela,-22.650244,-43.208156
151,Linha Guapimirim da SuperVia,Estação Parada Santa Guilhermina,-22.665836,-43.092736
152,Linha Guapimirim da SuperVia,Estação Parada Santa Dalila,-22.654022,-43.152528
153,Linha Belford Roxo da SuperVia,Estação Maracanã (SuperVia),-22.909272,-43.233817


In [3]:
data = []
with jsonlines.open('../data/input/BR_Rio_de_Janeiro_NULL_Rio_de_Janeiro_Zona_Sul_Copacabana.jsonl') as reader:
    for obj in reader:
        data.append(obj)

In [18]:
data[0]["link"]["href"]

'/imovel/aluguel-apartamento-1-quarto-com-elevador-copacabana-zona-sul-rio-de-janeiro-rj-47m2-id-2513808626/'

In [13]:
df = pd.json_normalize(data, sep='_')
df = df.groupby(["listing_portal", "link_href"]).head(1)
df.query("link_href == '/imovel/venda-apartamento-2-quartos-com-elevador-copacabana-zona-sul-rio-de-janeiro-rj-70m2-id-2512891177/'")

,medias,listing_displayAddressType,listing_amenities,listing_usableAreas,listing_constructionStatus,listing_listingType,listing_description,listing_title,listing_createdAt,listing_floors,listing_unitTypes,listing_nonActivationReason,listing_providerId,listing_propertyType,listing_unitSubTypes,listing_unitsOnTheFloor,listing_legacyId,listing_id,listing_portal,listing_unitFloor,listing_parkingSpaces,listing_updatedAt,listing_address_country,listing_address_zipCode,listing_address_geoJson,listing_address_city,listing_address_streetNumber,listing_address_level,listing_address_precision,listing_address_confidence,listing_address_stateAcronym,listing_address_source,listing_address_point_lon,listing_address_point_source,listing_address_point_lat,listing_address_ibgeCityId,listing_address_zone,listing_address_street,listing_address_locationId,listing_address_district,listing_address_name,listing_address_state,listing_address_neighborhood,listing_address_poisList,listing_address_complement,listing_address_pois,listing_address_valuableZones,listing_suites,listing_publicationType,listing_externalId,listing_bathrooms,listing_usageTypes,listing_totalAreas,listing_advertiserId,listing_advertiserContact_phones,listing_whatsappNumber,listing_bedrooms,listing_acceptExchange,listing_pricingInfos,listing_showPrice,listing_resale,listing_buildings,listing_capacityLimit,listing_status,account_id,account_name,account_licenseNumber,account_showAddress,account_legacyVivarealId,account_legacyZapId,accountLink_name,accountLink_href,accountLink_rel,link_data_city,link_data_zone,link_data_neighborhood,link_data_street,link_data_streetNumber,link_data_state,link_name,link_href,link_rel,account_logoUrl,listing_advertiserContact_chat
589,"[{'url': 'https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/10e7b877ff87ee12dff01fa7ed0241f9.jpg', 'type': 'IMAGE'}, {'url': 'https://youtu.be/y04SkqDVjQw', 'type': 'VID...",ALL,"[ELEVATOR, PLAYGROUND, INTERCOM, PARTY_HALL, KITCHEN_CABINETS, BUILTIN_WARDROBE, BALCONY]",[70],ConstructionStatus_NONE,USED,"Cód: GA20412. POSTO 6. PRÓXIMO A POUSADAS E METRÔ SÁ FERREIRA. PRÉDIO COM BICICLETARIO E SALÃO DE FESTAS. IMÓVEL COM SALA AMPLA, TODA EM CERÂMICA, 2 DORMITÓRIOS SENDO UM COM ARMÁRIO EMBUTIDO, 1 BA...",Rio de Janeiro - Apartamento Padrão - Copacabana,2021-03-05T16:39:48.700Z,[],[APARTMENT],NonActivationReason_NONE,48476,UNIT,[],0,,2512891177,ZAP,0,[1],2021-04-01T17:17:05.245Z,BR,22071060,,Rio de Janeiro,,STREET,GEOMETRIC_CENTER,VALID_STREET,RJ,CORREIOS,-43.19408,GOOGLE,-22.981043,,Zona Sul,Rua Saint Roman,BR>Rio de Janeiro>NULL>Rio de Janeiro>Zona Sul>Copacabana,,,Rio de Janeiro,Copacabana,"[BS:BRS 1 Sá Ferreira, BS:BRS 2 Djalma Ulrich, BS:BRS 3 Miguel Lemos, BS:BRS 1 Miguel Lemos, BS:Metrô na Superfície - General Osório, TS:Estação 1, TS:Estação 2, TS:Estação 3, TS:Estação 4, TS:Est...",,[],[],[0],STANDARD,GA20412,[1],[RESIDENTIAL],[70],ed62236d-a15e-2f5f-cafe-d21aae79dd2f,"[2138131248, 21965193012]",21965193012,[2],False,"[{'rentalInfo': {'period': 'MONTHLY', 'warranties': [], 'monthlyRentalTotalPrice': '1950'}, 'price': '1500', 'businessType': 'RENTAL', 'monthlyCondoFee': '450'}, {'price': '480000', 'businessType'...",True,False,0.0,[],ACTIVE,ed62236d-a15e-2f5f-cafe-d21aae79dd2f,GAFEN EMPREENDIMENTOS IMOBILIARIOS LTDA,04883-J-RJ,True,55445,906579,GAFEN EMPREENDIMENTOS IMOBILIARIOS LTDA,/imobiliaria/55445/,,Rio de Janeiro,Zona Sul,Copacabana,Rua Saint Roman,,Rio de Janeiro,Copacabana,/imovel/venda-apartamento-2-quartos-com-elevador-copacabana-zona-sul-rio-de-janeiro-rj-70m2-id-2512891177/,,NaN,NaN


In [14]:
df.loc[:, "media"] = df.medias.apply(
    lambda x: [i["url"].format(action="fit-in", width=870, height=653) for i in x]
)
df.loc[:, "created_at"] = pd.to_datetime(df.listing_createdAt)
df.loc[:, "updated_at"] = pd.to_datetime(df.listing_updatedAt)
df.loc[:, "usable_area"] = df.listing_usableAreas.str[0].astype(int)
df.loc[:, "floors"] = df.listing_floors.str[0].fillna(-1)
df.loc[:, "units_on_the_floor"] = df.listing_unitsOnTheFloor.fillna(-1)
df.loc[:, "unit_floor"] = df.listing_unitFloor.fillna(-1)
df.loc[:, "type_unit"] = df.listing_unitTypes.str[0]
df.loc[:, "bedrooms"] = df.listing_bedrooms.str[0]
df.loc[:, "bathrooms"] = df.listing_bathrooms.str[0]
df.loc[:, "suites"] = df.listing_suites.str[0].fillna(0)
df.loc[:, "parking_spaces"] = df.listing_parkingSpaces.str[0].fillna(0)

df = df.dropna(subset=["bedrooms", "bathrooms"])

In [16]:
df.listing_portal.value_counts()

GRUPOZAP    1539
ZAP          487
VIVAREAL     275
Name: listing_portal, dtype: int64

In [15]:
df.query("link_href == '/imovel/venda-apartamento-2-quartos-com-elevador-copacabana-zona-sul-rio-de-janeiro-rj-70m2-id-2512891177/'")

,medias,listing_displayAddressType,listing_amenities,listing_usableAreas,listing_constructionStatus,listing_listingType,listing_description,listing_title,listing_createdAt,listing_floors,listing_unitTypes,listing_nonActivationReason,listing_providerId,listing_propertyType,listing_unitSubTypes,listing_unitsOnTheFloor,listing_legacyId,listing_id,listing_portal,listing_unitFloor,listing_parkingSpaces,listing_updatedAt,listing_address_country,listing_address_zipCode,listing_address_geoJson,listing_address_city,listing_address_streetNumber,listing_address_level,listing_address_precision,listing_address_confidence,listing_address_stateAcronym,listing_address_source,listing_address_point_lon,listing_address_point_source,listing_address_point_lat,listing_address_ibgeCityId,listing_address_zone,listing_address_street,listing_address_locationId,listing_address_district,listing_address_name,listing_address_state,listing_address_neighborhood,listing_address_poisList,listing_address_complement,listing_address_pois,listing_address_valuableZones,listing_suites,listing_publicationType,listing_externalId,listing_bathrooms,listing_usageTypes,listing_totalAreas,listing_advertiserId,listing_advertiserContact_phones,listing_whatsappNumber,listing_bedrooms,listing_acceptExchange,listing_pricingInfos,listing_showPrice,listing_resale,listing_buildings,listing_capacityLimit,listing_status,account_id,account_name,account_licenseNumber,account_showAddress,account_legacyVivarealId,account_legacyZapId,accountLink_name,accountLink_href,accountLink_rel,link_data_city,link_data_zone,link_data_neighborhood,link_data_street,link_data_streetNumber,link_data_state,link_name,link_href,link_rel,account_logoUrl,listing_advertiserContact_chat,media,created_at,updated_at,usable_area,floors,units_on_the_floor,unit_floor,type_unit,bedrooms,bathrooms,suites,parking_spaces
589,"[{'url': 'https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/10e7b877ff87ee12dff01fa7ed0241f9.jpg', 'type': 'IMAGE'}, {'url': 'https://youtu.be/y04SkqDVjQw', 'type': 'VID...",ALL,"[ELEVATOR, PLAYGROUND, INTERCOM, PARTY_HALL, KITCHEN_CABINETS, BUILTIN_WARDROBE, BALCONY]",[70],ConstructionStatus_NONE,USED,"Cód: GA20412. POSTO 6. PRÓXIMO A POUSADAS E METRÔ SÁ FERREIRA. PRÉDIO COM BICICLETARIO E SALÃO DE FESTAS. IMÓVEL COM SALA AMPLA, TODA EM CERÂMICA, 2 DORMITÓRIOS SENDO UM COM ARMÁRIO EMBUTIDO, 1 BA...",Rio de Janeiro - Apartamento Padrão - Copacabana,2021-03-05T16:39:48.700Z,[],[APARTMENT],NonActivationReason_NONE,48476,UNIT,[],0,,2512891177,ZAP,0,[1],2021-04-01T17:17:05.245Z,BR,22071060,,Rio de Janeiro,,STREET,GEOMETRIC_CENTER,VALID_STREET,RJ,CORREIOS,-43.19408,GOOGLE,-22.981043,,Zona Sul,Rua Saint Roman,BR>Rio de Janeiro>NULL>Rio de Janeiro>Zona Sul>Copacabana,,,Rio de Janeiro,Copacabana,"[BS:BRS 1 Sá Ferreira, BS:BRS 2 Djalma Ulrich, BS:BRS 3 Miguel Lemos, BS:BRS 1 Miguel Lemos, BS:Metrô na Superfície - General Osório, TS:Estação 1, TS:Estação 2, TS:Estação 3, TS:Estação 4, TS:Est...",,[],[],[0],STANDARD,GA20412,[1],[RESIDENTIAL],[70],ed62236d-a15e-2f5f-cafe-d21aae79dd2f,"[2138131248, 21965193012]",21965193012,[2],False,"[{'rentalInfo': {'period': 'MONTHLY', 'warranties': [], 'monthlyRentalTotalPrice': '1950'}, 'price': '1500', 'businessType': 'RENTAL', 'monthlyCondoFee': '450'}, {'price': '480000', 'businessType'...",True,False,0.0,[],ACTIVE,ed62236d-a15e-2f5f-cafe-d21aae79dd2f,GAFEN EMPREENDIMENTOS IMOBILIARIOS LTDA,04883-J-RJ,True,55445,906579,GAFEN EMPREENDIMENTOS IMOBILIARIOS LTDA,/imobiliaria/55445/,,Rio de Janeiro,Zona Sul,Copacabana,Rua Saint Roman,,Rio de Janeiro,Copacabana,/imovel/venda-apartamento-2-quartos-com-elevador-copacabana-zona-sul-rio-de-janeiro-rj-70m2-id-2512891177/,,NaN,NaN,"[https://resizedimgs.zapimoveis.com.br/fit-in/870x653/vr.images.sp/10e7b877ff87ee12dff01fa7ed0241f9.jpg, https://youtu.be/y04SkqDVjQw, https://resizedimgs.zapimoveis.com.br/fit-in/870x653/vr.image...",2021-03-05 16:39:48.700000+00:00,2021-04-01 17:17:05.245000+00:00,70,-1.0,0,

In [7]:
df.loc[:, "created_at"] = pd.to_datetime(df.listing_createdAt)
df.loc[:, "updated_at"] = pd.to_datetime(df.listing_updatedAt)
df.loc[:, "usable_area"] = df.listing_usableAreas.str[0].astype(int)
df.loc[:, "floors"] = df.listing_floors.str[0].fillna(-1)
df.loc[:, "units_on_the_floor"] = df.listing_unitsOnTheFloor.fillna(-1)
df.loc[:, "unit_floor"] = df.listing_unitFloor.fillna(-1)
df.loc[:, "type_unit"] = df.listing_unitTypes.str[0]
df.loc[:, "bedrooms"] = df.listing_bedrooms.str[0]
df.loc[:, "bathrooms"] = df.listing_bathrooms.str[0]
df.loc[:, "suites"] = df.listing_suites.str[0].fillna(0)
df.loc[:, "parking_spaces"] = df.listing_parkingSpaces.str[0].fillna(0)

df = df.dropna(subset=["bedrooms", "bathrooms"])

In [8]:
df.query("link_href.str.contains('2514710758')")

,medias,listing_displayAddressType,listing_amenities,listing_usableAreas,listing_constructionStatus,listing_listingType,listing_description,listing_title,listing_createdAt,listing_floors,listing_unitTypes,listing_nonActivationReason,listing_providerId,listing_propertyType,listing_unitSubTypes,listing_unitsOnTheFloor,listing_legacyId,listing_id,listing_portal,listing_unitFloor,listing_parkingSpaces,listing_updatedAt,listing_address_country,listing_address_zipCode,listing_address_geoJson,listing_address_city,listing_address_streetNumber,listing_address_level,listing_address_precision,listing_address_confidence,listing_address_stateAcronym,listing_address_source,listing_address_point_lon,listing_address_point_source,listing_address_point_lat,listing_address_ibgeCityId,listing_address_zone,listing_address_street,listing_address_locationId,listing_address_district,listing_address_name,listing_address_state,listing_address_neighborhood,listing_address_poisList,listing_address_complement,listing_address_pois,listing_address_valuableZones,listing_suites,listing_publicationType,listing_externalId,listing_bathrooms,listing_usageTypes,listing_totalAreas,listing_advertiserId,listing_advertiserContact_phones,listing_whatsappNumber,listing_bedrooms,listing_acceptExchange,listing_pricingInfos,listing_showPrice,listing_resale,listing_buildings,listing_capacityLimit,listing_status,account_id,account_name,account_licenseNumber,account_showAddress,account_legacyVivarealId,account_legacyZapId,accountLink_name,accountLink_href,accountLink_rel,link_data_city,link_data_zone,link_data_neighborhood,link_data_street,link_data_streetNumber,link_data_state,link_name,link_href,link_rel,account_logoUrl,listing_advertiserContact_chat,created_at,updated_at,usable_area,floors,units_on_the_floor,unit_floor,type_unit,bedrooms,bathrooms,suites,parking_spaces
588,"[{'url': 'https://resizedimgs.zapimoveis.com.br/{action}/{width}x{height}/vr.images.sp/bcee4e96dc847950904afce7a63f2b16.jpg', 'type': 'IMAGE'}, {'url': 'https://resizedimgs.zapimoveis.com.br/{acti...",ALL,"[ELEVATOR, INTERCOM, PETS_ALLOWED, FURNISHED]",[30],ConstructionStatus_NONE,USED,"Alugo kitnet/conjugado em Copacabana com 30 m², com quarto todo mobiliado, cama de casal, guarda roupas de 2 portas, TV, mesa e cadeiras, ar condicionado. possui 1 banheiro com box blindex.. Imíov...",Kitnet/Conjugado mobiliado com ar para aluguel com 30 m² e 1 quarto,2021-03-24T16:09:00.925Z,[10],[KITNET],NonActivationReason_NONE,,UNIT,[],4,,2514710758,GRUPOZAP,9,[0],2021-03-31T14:45:35.791Z,BR,22051001,,Rio de Janeiro,793,STREET,ROOFTOP,VALID_STREET,RJ,CORREIOS,-43.192441,GOOGLE,-22.97667,,Zona Sul,Rua Barata Ribeiro,BR>Rio de Janeiro>NULL>Rio de Janeiro>Zona Sul>Copacabana,,,Rio de Janeiro,Copacabana,"[BS:BRS 3 Miguel Lemos, BS:BRS 1 Xavier da Silveira, BS:BRS 2 Bolívar, BS:BRS 3 Bolivar, BS:BRS 1 Miguel Lemos, TS:BRS 3 Miguel Lemos, TS:BRS 1 Xavier da Silveira, TS:Estação Cantagalo, TS:BRS 2 B...",,[],[],[],STANDARD,Barata793,[1],[RESIDENTIAL],[30],ee6eb1cc-1d8f-29a5-e63f-d450e9edfb9d,"[21984522698, 21971723664]",21971723664,[1],False,"[{'rentalInfo': {'period': 'MONTHLY', 'warranties': ['INSURANCE_GUARANTEE', 'GUARANTEE_LETTER', 'GUARANTOR', 'CAPITALIZATION_BONDS', 'SECURITY_DEPOSIT'], 'monthlyRentalTotalPrice': '1720'}, 'yearl...",True,False,0.0,[],ACTIVE,ee6eb1cc-1d8f-29a5-e63f-d450e9edfb9d,Agita Imóveis,,False,646549,0,Agita Imóveis,/imobiliaria/646549/,,Rio de Janeiro,Zona Sul,Copacabana,Rua Barata Ribeiro,793,Rio de Janeiro,Copacabana,/imovel/aluguel-quitinete-1-quarto-mobiliado-copacabana-zona-sul-rio-de-janeiro-rj-30m2-id-2514710758/,,NaN,NaN,2021-03-24 16:09:00.925000+00:00,2021-03-31 14:45:35.791000+00:00,30,10.0,4,9,KITNET,1,1,0.0,0.0
1281,"[{'url': 'https://resizedimgs.vivareal.com/{action}/{width}x{height}/vr.images.sp/bcee4e96dc847950904afce7a63f2b16.jpg', 'type': 'IMAGE'}, {'url': 'https://resizedimgs.vivareal.com/{action}/{width...",ALL,"[FURNISHED, ELEVATOR, PETS_ALLOWED, INTERCOM]",[30],Construction

In [ ]:
df = pd.concat(
        [
            df,
            df.listing_pricingInfos.apply(
                lambda x: [
                    {
                        "price": p.get("price", 0),
                        "condo_fee": p.get("monthlyCondoFee", 0),
                    }
                    for p in x
                    if p["businessType"] == "RENTAL"
                ][0]
            )
            .apply(pd.Series)
            .astype(int),
        ],
        axis=1,
    ).assign(total_fee=lambda x: x["price"] + x["condo_fee"])

In [ ]:
df = pd.json_normalize(data, sep="_")
df.loc[:, "media"] = df.medias.apply(
    lambda x: [i["url"].format(action="fit-in", width=870, height=653) for i in x]
)
df.loc[:, "created_at"] = pd.to_datetime(df.listing_createdAt)
df.loc[:, "updated_at"] = pd.to_datetime(df.listing_updatedAt)
df.loc[:, "usable_area"] = df.listing_usableAreas.str[0].astype(int)
df.loc[:, "floors"] = df.listing_floors.str[0].fillna(-1)
df.loc[:, "units_on_the_floor"] = df.listing_unitsOnTheFloor.fillna(-1)
df.loc[:, "unit_floor"] = df.listing_unitFloor.fillna(-1)
df.loc[:, "type_unit"] = df.listing_unitTypes.str[0]
df.loc[:, "bedrooms"] = df.listing_bedrooms.str[0]
df.loc[:, "bathrooms"] = df.listing_bathrooms.str[0]
df.loc[:, "suites"] = df.listing_suites.str[0].fillna(0)
df.loc[:, "parking_spaces"] = df.listing_parkingSpaces.str[0].fillna(0)

In [ ]:
df = pd.concat(
    [
        df,
        df.listing_pricingInfos.apply(
            lambda x: [
                {
                    "price": p.get("price", 0),
                    "condo_fee": p.get("monthlyCondoFee", 0),
                }
                for p in x
                if p["businessType"] == "RENTAL"
            ][0]
        )
        .apply(pd.Series)
        .astype(int),
    ],
    axis=1,
).assign(total_fee=lambda x: x["price"] + x["condo_fee"])

fl_zap = df["listing_portal"] == "GRUPOZAP"
df.loc[fl_zap, "origin"] = "zapimoveis"
df.loc[~fl_zap, "origin"] = "vivareal"

df.loc[:, "url"] = df[["origin", "link_href"]].apply(
    lambda x: "https://www.{}.com.br{}".format(*x), axis=1
)

In [ ]:
df[df.bedrooms.isna()].url

In [ ]:
df[df.url.str.contains("2495611515")]

In [ ]:
df.listing_bathrooms

In [ ]:
df.listing_usableAreas.str[0]

In [ ]:
t = df[df.listing_pricingInfos.apply(lambda x: len(x)) == 2].listing_pricingInfos[41]
t

In [ ]:
t[1]

In [ ]:
[a for a in t if a["businessType"] == 'RENTAL']

In [ ]:
df

In [ ]:
pd.concat(
        [
            df,
            df.listing_pricingInfos.apply(
                lambda x: [
                    {
                        "price": p.get("price", 0),
                        "condo_fee": p.get("monthlyCondoFee", 0),
                    }
                    for p in x
                    if p["businessType"] == "RENTAL"
                ][0]
            ).apply(pd.Series).astype(int)
        ],
        axis=1,
    ).dtypes

In [ ]:
df.listing_pricingInfos.apply(lambda x: [{"price": p.get("price", 0), "condo_fee": p.get("monthlyCondoFee", 0)} for p in x if p["businessType"] == 'RENTAL'][0]).apply(pd.Series).astype(int)

In [ ]:
df.listing_unitsOnTheFloor

In [ ]:
df.listing_pricingInfos[0]

In [ ]:
df.medias.apply(lambda x: [i["url"].format(action="fit-in", width=870, height=653) for i in x])

In [ ]:
def get_min_dist(lat, lng):
    if not pd.isna(lat):
        X = df_metro.copy()
        X["distance"] =  X[["lat", "lng"]].apply(lambda x: distance((x[0], x[1]), (lat, lng)).m, axis=1)
        return X.sort_values("distance").reset_index().loc[0,['linha', 'estacao', 'distance']].tolist()
        #return X.sort_values("distance").reset_index()
    else:
        return "", "", None

In [ ]:
df[['lat', 'lng','condominio']] = df[['lat', 'lng','condominio']].apply(pd.to_numeric)

In [ ]:
get_min_dist(-22.925214,-43.230616)

In [ ]:
df[['linha', 'estacao', 'distance']] = df[["lat", "lng"]].apply(lambda x: get_min_dist(x[0], x[1]), axis=1).apply(pd.Series)

In [ ]:
df[df.url == "https://www.vivareal.com.br/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-65m2-aluguel-RS1550-id-92895851/?__vt=lgpd:b"]

In [ ]:
df[~df.lat.isna()]

In [ ]:
df.groupby("page")["id"].count()

In [ ]:
df.groupby("tipo")["id"].count()

In [ ]:
df = df[df.tipo == "Aluguel"]
df.head(3)

In [ ]:
df["preco_total"] = df["preco"] + df["condominio"]

In [ ]:
df.head(3)